# 서울시 체육시설과 소득 간 비교 분석
서울데이터허브(전국 체육시설 현황·표준데이터)를 기반으로, 서울특별시 자치구별 시설 유형 분포를 시각화하고 구별 소득과의 관계를 간단히 살펴봅니다.

In [ ]:
!pip install pandas matplotlib seaborn openpyxl

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
sns.set_theme(style='whitegrid')

In [2]:
facility_csv = 'data/전국체육시설 현황(표준데이터).csv'
facility_df = pd.read_csv(facility_csv, encoding='cp949', low_memory=False)
facility_df = facility_df.rename(columns=str.strip)
facility_df.head(3)

,이용단체명,시설생성기준일자,등록신고일자,준공일자,휴업일자,폐업일자,국가체육시설여부,시설명,시설구분명,순번,시설주소,시설상세주소,시설코드,업종명,시설유형명,시설상태명,시설도로명우편번호,시설도로명주소,시설도로명상세주소,우편번호,경도,위도,시설전화번호,자율점검대상여부,등록일시,수정일시,내진설계여부,시설홈페이지,시도,시군구,도로명시도명,도로명시군구명,도로명읍면동명,도로명리명,시설운영형태코드,소유주체명,소유주체시도명,소유주체시군구명,담당자부서명,담당자전화번호,실내야외구분,관람좌석수,관람수용인원수,시설총면적,생활오픈여부,생활체육시설명
0,NaN,20210310,NaN,NaN,NaN,20250620.0,N,위드미 댄스,신고,1,경기도 고양시 덕양구 화정동 968 비젼타워21,1005호,E1DE682A80811DA7E71A53B2F1D45637,무도학원업,무도학원,폐업,10497,경기도 고양시 덕양구 화중로 100(화정동),NaN,10497.0,126.831467,37.636138,NaN,N,2021-08-11,2025-06-21,NaN,NaN,경기도,고양시,경기도,고양시 덕양구,화정동,NaN,자체운영,NaN,경기도,고양시,NaN,NaN,없음,NaN,NaN,88.0,NaN,NaN
1,NaN,20210309,NaN,NaN,NaN,NaN,N,히키스 캐롬 클럽,신고,2,경기도 고양시 덕양구 신원동 633,NaN,97E7959ABA7ED5F870AF36079FC6E98F,당구장업,당구장,정상운영,10571,경기도 고양시 덕양구 신원로 10(신원동),NaN,10571.0,126.890319,37.661865,NaN,Y,2021-08-11,2024-10-31,NaN,NaN,경기도,고양시,경기도,고양시,신원동,NaN,NaN,NaN,NaN,NaN,NaN,NaN,없음,NaN,NaN,248.0,NaN,NaN
2,NaN,20200929,NaN,NaN,NaN,20240621.0,N,빅이닝 고양 볼파크,신고,3,경기도 고양시 덕양구 오금동 379-1,NaN,0CC77977D1C368EB9B82BB3B85F2F7B7,야구장업,야구장,폐업,10574,경기도 고양시 덕양구 통일로396번길 78-25(오금동),(오금동),10574.0,0.000000,0.000000,NaN,N,2021-08-11,2024-10-31,NaN,NaN,경기도,고양시,경기도,고양시,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,없음,NaN,NaN,10308.0,NaN,NaN


In [4]:
facility_excel = 'data/전국체육시설 현황(표준데이터).xlsx'
pd.read_excel(facility_excel, engine='openpyxl', nrows=3)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

### 서울시 자치구 필터링
서울특별시에 있는 시설만 남기고 자치구와 시설 유형 중심으로 분석합니다.

In [ ]:
seoul_df = facility_df[facility_df['시도'] == '서울특별시'].copy()
seoul_df['시군구'] = seoul_df['시군구'].str.strip()
seoul_df[['시설명', '시설유형명', '시군구']].head(3)


In [ ]:
district_type_counts = (
    seoul_df.groupby(['시군구', '시설유형명'])
    .size()
    .reset_index(name='count')
)
district_totals = (
    district_type_counts
    .groupby('시군구', as_index=False)['count']
    .sum()
    .rename(columns={'count': 'facility_count'})
)
district_totals.sort_values('facility_count', ascending=False).head(5)


In [ ]:
top_types = (
    district_type_counts.sort_values(['시군구', 'count'], ascending=[True, False])
    .groupby('시군구')
    .head(5)
)
stacked = (
    top_types
    .pivot(index='시군구', columns='시설유형명', values='count')
    .fillna(0)
)
stacked = stacked.reindex(sorted(stacked.index))
plt.figure(figsize=(16, 8))
stacked.plot(kind='bar', stacked=True, colormap='tab20', width=0.9)
plt.title('서울 구별 주요 시설 유형 분포 (상위 5개 유형 기준)')
plt.xlabel('자치구')
plt.ylabel('시설 개수')
plt.xticks(rotation=45, ha='right')
plt.legend(title='시설유형명', bbox_to_anchor=(1.02, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
income_values = [
    ('종로구', 4420), ('중구', 4300), ('용산구', 5200), ('성동구', 4700), ('광진구', 4300),
    ('동대문구', 4100), ('중랑구', 4000), ('성북구', 4200), ('강북구', 3900), ('도봉구', 3800),
    ('노원구', 4000), ('은평구', 4100), ('서대문구', 4500), ('마포구', 4700), ('양천구', 4400),
    ('강서구', 4200), ('구로구', 4100), ('금천구', 3900), ('영등포구', 4200), ('동작구', 4300),
    ('관악구', 4000), ('강남구', 5500), ('서초구', 5400), ('송파구', 5200), ('강동구', 4600)
]
income_df = pd.DataFrame(income_values, columns=['시군구', '연간_소득_만원']).sort_values('시군구').reset_index(drop=True)
income_df

In [ ]:
district_insights = district_totals.merge(income_df, how='left', on='시군구')
district_insights['시설당_소득비율'] = district_insights['facility_count'] / district_insights['연간_소득_만원']
district_insights.sort_values('시설당_소득비율', ascending=False)


In [ ]:
plt.figure(figsize=(10, 6))
sns.regplot(
    data=district_insights,
    x='연간_소득_만원',
    y='facility_count',
    scatter_kws={'s': 70, 'alpha': 0.8},
    line_kws={'color': 'darkorange'}
)
for _, row in district_insights.iterrows():
    plt.text(row['연간_소득_만원'], row['facility_count'] + 8, row['시군구'], fontsize=8, ha='center')
plt.title('서울 구별 종합 시설 수와 연간 소득의 관계')
plt.xlabel('평균 연간 소득 (만원)')
plt.ylabel('자치구 시설 수')
plt.tight_layout()
plt.show()


### 정책 과제 및 제안
1. 소득 대비 시설 수가 낮은 자치구(예: 도봉구, 강북구)에 지역체육센터나 생활체육 프로그램을 집중 배치해 접근성을 높입니다.2. 고소득 자치구에는 공공 시설을 거점으로 하는 주민 참여형 프로그램을 운영하고, 정기 점검을 통해 시설 균형 유지 여부를 확인합니다.3. 구별 시설·소득 지표를 분기 단위로 업데이트하여 `시설당_소득비율`이 과하게 떨어지는 지역을 빠르게 발견하고 예산을 탄력적으로 조정합니다.